#SEMPIE - SEM Image Pore Extractor | <img src="logo/ceitec_logo.png" /> | <img src="logo/bdalab.jpg" /> | <img src="logo/vut.jpg" /> 

In [13]:
# !pip install ipyfilechooser
# !pip install seaborn
# !pip install plotly
# !pip install -U kaleido
# !pip install psutil
# !pip install xlsxwriter
# !pip install ipyvuetify
# !pip install voila-material
# !jupyter nbextension enable --py --sys-prefix ipyvuetify
# !pip install voila-gridstack
%config IPCompleter.greedy=True
import os

import numpy as np
import pandas as pd
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

import cv2
import math
import warnings
import time    

import ipyvuetify as v
import ipywidgets as widgets
from ipyfilechooser import FileChooser
from ipywidgets.widgets import interact
from IPython.display import display, clear_output


from ipywidgets import Button, Layout, Box, HBox
from tkinter import Tk, filedialog


<!-- ### ImageProcessing Class -->


In [14]:
class ImageProcessing():
  """ Class implementing all image analysis functions"""

  KNOWN_RESOLUTIONS = {
      1024: 888,
      1280: 1110,
      2048: 1776,
      1280: 1430,
  }

  DEFAULT_CROP_COEFICIENT = 0.1351

  DEFAULT_WIDTH_RES = 1024
  DEFAULT_HEIGHT_RES = 768

  DEFAULT_MIN_PORE = 900

  DEFAULT_GAUSS_KERNEL = 15
  DEFAULT_LAPLACIAN_KERNEL = 3
  
  DEFAULT_PIXELIZATION = 256  

  DEFAULT_LOW_THRESHOLD = 50
  DEFAULT_HIGH_THRESHOLD = 255

  DEFAULT_DILATE_ITERATIONS = 1

  @staticmethod
  def crop_label(image, 
                 crop_coeficient=DEFAULT_CROP_COEFICIENT):
    """
    Crop out the information label from the SEM image. 
    We established the crop_coeficient regarding the SEM used on CEITEC (13.15%)

    :param image:           Input image
    :param crop_coeficient: Size of the image part to be croped out from the bottom of the input image. DEFAULT = 0.1351

    :return: Cropped image 
    """

    # Check input
    if image is None:
      raise ValueError("\nCROP_LABEL: No image on input")
    # crop must not be higher than 1
    if crop_coeficient >= 1:
      raise ValueError("\nCROP_LABEL: crop coeficient must be lower than 1, otherwise entire image will be cropped.")
    # if crop is more than 25% is strange, raise warning at least
    if crop_coeficient > 0.25:
      warnings.warn("\nCROP_LABEL: You are going to crop more than 25% of the image, please be sure!")

    # Get size of image
    height, width = image.shape[:2]

    # Compute crop size from bottom
    crop_size = math.ceil(height*crop_coeficient)

    # Check is image resolution is known
    if ImageProcessing.KNOWN_RESOLUTIONS.get(width, 0) == 0:
      warnings.warn(f"\nUnknown resolution, please pay attention to image label cropping.\n " 
                    f"Original image resolution is {width} x {height}.\n "
                    f"Image after cropping will have {width} x {height - crop_size}.\n")
      
    elif ImageProcessing.KNOWN_RESOLUTIONS.get(width) != height:
      warnings.warn(f"\nUnknown height, for this image resolution. Please consider change of the crop_coeficient.\n" 
                    f"Original image resolution is {width} x {height}.\n"
                    f"Known image resolution is {width} x {ImageProcessing.KNOWN_RESOLUTIONS.get(width)}.\n"
                    f"Image after cropping will have {width} x {height - crop_size}.\n")
      
    # Crop image
    x=0
    y=0
    h=crop_size
    w=width
    crop_image = image[x:x-h, y:w]

    # Return
    return crop_image

  @staticmethod
  def resize(image, 
             width=DEFAULT_WIDTH_RES, 
             height=DEFAULT_HEIGHT_RES):
    """
    Resize the input image to one pre-defined dimension

    :param image:   Input image
    :param width:   Width of output image. DEFAULT = 1024
    :param height:  Height of output image. DEFAULT = 768

    :return: Resized image 
    """

    # Check input
    if image is None:
      raise ValueError("\nRESIZE: No image on input")

    # Define new dimension
    dim = (width, height)

    # Resize Image
    resized_image = cv2.resize(image, dim)

    # Check resized dimensions
    res_height, res_width = resized_image.shape[:2]
    if not res_height == height or not res_width == width:
      raise ValueError(f"\nRESIZE: Image does not have the right resolution: {res_width} x {res_height}\n")
    
    return resized_image

  @staticmethod
  def normalize_image(image):
    """
    Normalize image

    :param image: Input preprocessed image

    :return: Normalized image
    """

    # Check input
    if image is None:
      raise ValueError("\nNorm: No image on input")

    # preform normalization
    norm_image = cv2.normalize(src=image, 
                               dst=None, 
                               alpha=0, 
                               beta=255, 
                               norm_type=cv2.NORM_MINMAX, 
                               dtype=cv2.CV_8U)
    return norm_image

  @staticmethod
  def blur_image(image, 
                 kernel_size=None):
    """
    Blur image

    :param image:       Input normlaized image
    :param kernel_size: Size of gaussian kernel

    :return: Blured image
    """

    # Check input
    if image is None:
      raise ValueError("\nGauss: No image on input")

    if kernel_size is None:
      kernel_size=ImageProcessing.DEFAULT_GAUSS_KERNEL

    # Blur image
    blured = cv2.GaussianBlur(image, (kernel_size, kernel_size), 0)

    return blured

  @staticmethod
  def laplacian_image(image,
                      kernel_size=None):
    """
    Perform Laplacian operation on Image

    :param image:       Input normalized image
    :param kernel_size: Size of Laplacian kernel

    :return: Laplacian image
    """

    # Check input
    if image is None:
      raise ValueError("\nLaplacian: No image on input")

    if kernel_size is None:
      kernel_size=ImageProcessing.DEFAULT_LAPLACIAN_KERNEL

    # Apply Laplacian operation
    laplacian_image = cv2.Laplacian(image, cv2.CV_16S, kernel_size)

    return laplacian_image

  @staticmethod
  def sharpen_image(normal_image, 
                    blured_image):
    """
    Sharpen image

    :param normal_image:  Input normlaized image
    :param blured_image:  Input blured image

    :return: Sharped image
    """

    # Check input
    if normal_image is None:
      raise ValueError("\nSharpen: No normal_image on input")
    if blured_image is None:
      raise ValueError("\nSharpen: No blured_image on input")

    # Sharpen image
    sharped_image = cv2.addWeighted(normal_image, 1.5, blured_image, -1, 0)

    return sharped_image

  @staticmethod
  def create_binary_mask(image,
                         low_threshold=None,
                         high_threshold=None):
    """
    Create binary mask from image

    :param image:           Input sharped image
    :param low_threshold:   Low threshold value
    :param high_threshold:  High threshold value  

    :return: binary mask
    """
    
    # Check input
    if image is None:
      raise ValueError("\nMask: No image on input")

    if low_threshold is None:
      low_threshold = ImageProcessing.DEFAULT_LOW_THRESHOLD
    if high_threshold is None:
      high_threshold = ImageProcessing.DEFAULT_HIGH_THRESHOLD
    
    # Create mask
    _, binary_mask = cv2.threshold(image, 
                                   low_threshold, 
                                   high_threshold, 
                                   cv2.THRESH_BINARY)
    return binary_mask

  @staticmethod
  def dilate_image(binary_mask,
                   iterations=None):
    """
    Dilate image

    :param binary_mask:  Input binary mask
    :param iterations:   Number of iterations for dilatation process

    :return: dilated image
    """
    # Check input
    if binary_mask is None:
      raise ValueError("\nDilate: No image on input")

    if iterations is None:
      iterations = ImageProcessing.DEFAULT_DILATE_ITERATIONS

    # Dilate binary mask
    dilated_image = cv2.dilate(binary_mask, None, iterations=iterations)

    return dilated_image

  @staticmethod
  def find_contours(image,
                    min_area=None,
                    inverse=False):
    """
    Find the contours and compute areas

    :param image:     Input image
    :param min_area:  Minimum area of the countour to be counted in, DEFAULT = 150
    :param inverse:   Set tu True if inverse input image (e.g. for pore analysis) 

    :return: Two List -> list of contours, list of countour's areas 
    """

    # Check input
    if image is None:
      raise ValueError("\nFindCountours: No image on input")

    if min_area is None:
      min_area = ImageProcessing.DEFAULT_MIN_PORE

    # Inverse is True
    if inverse:
      image = cv2.bitwise_not(image)
    
    # Find contours 
    all_contours, hierarchy = cv2.findContours(image, 
                                          cv2.RETR_TREE, 
                                          cv2.CHAIN_APPROX_SIMPLE)
    
    contours=[]
    areas=[]
    # Go over, store areas and remove smaler than minimum
    for cont in all_contours:    

      # Compute area
      cont_area = cv2.contourArea(cont) 

      # Filter out small objects
      if cont_area < min_area:
        # Skip contour
        continue

      # Store contour 
      contours.append(cont)
      areas.append(cont_area)
    
    # Return filtered contours
    return contours, areas

  @staticmethod
  def pixelite_image(image, 
                     pixel_size=None):
    """
    Pixelite the input Image

    :param image:       Input image
    :param pixel_size:  Size of the resulted one pixel DEFAULT = 256

    :return: pixelited image
    """

    # Check input
    if image is None:
      raise ValueError("\nPixelite: No image on input")
    if pixel_size is None:
      pixel_size=ImageProcessing.DEFAULT_PIXELIZATION
    
    # Get input size
    height, width = image.shape[:2]

    # Desired "pixelated" size
    w, h = (pixel_size, pixel_size)

    # Resize input to "pixelated" size
    temp = cv2.resize(image, (w, h), interpolation=cv2.INTER_LINEAR)

    # Initialize output image
    pixelite_image = cv2.resize(temp, (width, height), interpolation=cv2.INTER_NEAREST)

    return pixelite_image    

<!-- ### PoreAnalyzator Class -->

In [15]:
class PoreAnalyzator():
  """
  Main Class implementing the analysis and extraction of the pores from the SEM Image
  """
  MIN_PORE_DELIMETER = 8
  DEFAULT_LOW_THRESHOLD = 50
  DEFAULT_ALPHA = 0.2
    
  pore_diameters = []
  pore_image = None
  analysis_image = None

  def __init__(self):
    # Set Image Processor
    self.processor = ImageProcessing()

  def extract_pores(self,
                    image,
                    view_field,
                    gauss_kernel_size=None,
                    laplacian=False,
                    laplacian_kernel_size=None,
                    pixelite=False,
                    pixel_size=None,
                    low_threshold=None,
                    high_threshold=None,
                    dilatation_iteration=None,
                    min_pore_delimeter=None,
                    inverse_contours=True,                    
                    pore_color=None,
                    pore_aplha=None):
    """
    Extract pores from the input image.
    Main function of PoreAnalyzator which performes:
     - image preprocessing (crop and resize)
     - pore extraction
     - pore area calculation

    :param image:                   Input image
    :param view_field:              Dimension in milimiters of X asis of the image. This value is indicated in SEM image label
    :param gauss_kernel_size:       [OPTIONAL] Size of gaussian kernel DEFAULT=15
    :param laplacian:               [OPTIONAL] Set to true if want to use laplacian operator
    :param laplacian_kernel_size:   [OPTIONAL] Size of laplacian kernel DEFAULT=3
    :param pixelite:                [OPTIONAL] Set to true if want to use pixelization
    :param pixel_size:              [OPTIONAL] Size of the one pixel for pixelization DEFAULT=256
    :param low_threshold:           [OPTIONAL] Low threshold value for binary mask DEFAULT=50    
    :param high_threshold:          [OPTIONAL] High threshold value for binary mask DEFAULT=255
    :param dilatation_iteration:    [OPTIONAL] Number of iterations for dilatation process DEFAULT=1
    :param min_pore_delimeter:      [OPTIONAL] Minimum delimeter of the pore to be counted in DEFAULT=8
    :param inverse_contours:        [OPTIONAL] Set tu True if inverse input image (e.g. for pore analysis) DEFAULT=True
    :param pore_color:              [OPTIONAL] Color of the pores in final image
    :param pore_aplha:              [OPTIONAL] Overlay aplha level of the pores on original image DEFAULT=0.2

    :return pore_image:     image with identified pores
    :return pore_diameters: list of identified pore diameters in micrometer
    """

    # Check input
    if image is None:
      raise ValueError("\n_prepare_image: No image on input")
    if view_field is None:
      raise ValueError("\n_prepare_image: No view_field on input")
    if min_pore_delimeter is None:
      min_pore_delimeter = self.MIN_PORE_DELIMETER

    # Convert image to gray scale
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Preprocess_image 
    prepared_image = self.get_preprocessed_image(gray_image)

    # Get micrometer per pixel unit
    um_per_pixel = self.get_micro_meter_per_pixel(image, 
                                                  view_field)

    # Normalize image
    normalized_image = self.get_normalized_image(prepared_image)

    # Blure image
    blured_image = self.get_blured_image(normalized_image, 
                                         kernel_size=gauss_kernel_size)
    
    # If Laplacian operation is wanted apply it before Sharpen
    if laplacian:
      # Apply Laplace operator
      laplacian_image = self.get_laplacian_image(blured_image,
                                                 kernel_size=laplacian_kernel_size)
      # Apply sharpen on laplacian
      sharped_image = self.get_sharped_image(normalized_image, 
                                             cv2.convertScaleAbs(laplacian_image))    
    else:
      # Apply sharpen on blured
      sharped_image = self.get_sharped_image(normalized_image, 
                                             blured_image)
    
    # If Pixelite operation is wanted apply it before Mask
    if pixelite:
      # Apply pixelation
      pixelate_image = self.get_pixelited_image(sharped_image,
                                                pixel_size=pixel_size)
      # Create binary mask from pixelite image
      binary_mask = self.get_mask(pixelate_image,
                                  low_threshold=low_threshold,
                                  high_threshold=high_threshold)
    else:
      # Create binary mask from sharpen image
      binary_mask = self.get_mask(sharped_image,
                                  low_threshold=low_threshold,
                                  high_threshold=high_threshold)
      
    # Dilate mask
    dilated_image = self.get_dilated_image(binary_mask,
                                           iterations=dilatation_iteration)
    
    # Extract pore countours and pore areas
    # Transform delimeter to circle area
    min_pore_area = (math.pi * math.pow(min_pore_delimeter,2) / 4)
    # Transform minimal area to pixels
    min_pore_area = min_pore_area / um_per_pixel
    # Extract
    pore_contours, pore_areas = self.get_contours(dilated_image, 
                                                  min_pore_area=min_pore_area, 
                                                  inverse=inverse_contours)

    # Get final image with overlayed pores
    self.pore_image = pore_analyzator._overlay_pore_mask_to_original(prepared_image, 
                                                                     pore_contours, 
                                                                     image, 
                                                                     alpha=pore_aplha,
                                                                     pore_color=pore_color)
    # Convert pore areas to delimeters in micrometers
    self.pore_diameters = self._convert_pixels_areas_to_diameter(pore_areas,
                                                                 um_per_pixel)
    # return
    return self.pore_image, self.pore_diameters


  def get_preprocessed_image(self, 
                             image):
    """
    Get preprocessed image (croped and resized)

    :param image: Input image

    :return: Preprocessed image
    """

    return self._prepare_image(image)


  def get_micro_meter_per_pixel(self, 
                                image, 
                                view_field):    
    """
    Get unit of mircometer per pixel from the image and its view field

    :param image:       Input image
    :param view_field:  Dimension in milimiters of X asis of the image. This value is indicated in SEM image label

    :return: micrometer per pixel unit
    """

    return self._get_micro_meter_per_pixel(image, view_field)


  def get_normalized_image(self,
                           image):
    """
    Get normalized image

    :param image: Input preprocessed image

    :return: Normalized image
    """

    return self.processor.normalize_image(image)


  def get_blured_image(self,
                       image,
                       kernel_size=None):
    """
    Get blured image

    :param image: Input normalized image
    :param kernel_size  Size of gaussian kernel

    :return: Blured image
    """

    return self.processor.blur_image(image, kernel_size)

  def get_laplacian_image(self,
                          image,
                          kernel_size=None):
    """
    Get Laplacian image

    :param image: Input normalized image
    :param kernel_size  Size of laplacian kernel

    :return: Laplacian image
    """

    return self.processor.laplacian_image(image, kernel_size)

  def get_sharped_image(self,
                        normal_image,
                        blured_image):
    """
    Sharpen image

    :param normal_image:  Input normlaized image
    :param blured_image:  Input blured image

    :return: Sharped image
    """
    
    return self.processor.sharpen_image(normal_image, blured_image)

  def get_pixelited_image(self,
                          image,
                          pixel_size=None):
    """
    Pixelite the input Image

    :param image:       Input image
    :param pixel_size:  Size of the resulted one pixel DEFAULT = 256

    :return: Pixelated Image
    """

    return self.processor.pixelite_image(image, pixel_size=pixel_size)

  def get_mask(self,
               image,
               low_threshold=None,
               high_threshold=None):
    """
    Create binary mask from image

    :param image:           Input sharped image
    :param low_threshold:   Low threshold value
    :param high_threshold:  High threshold value  

    :return: binary mask
    """

    return self.processor.create_binary_mask(image,
                                             low_threshold=low_threshold,
                                             high_threshold=high_threshold)
  def get_dilated_image(self,
                        image,
                        iterations=None):
    """
    Dilate image

    :param binary_mask:  Input binary mask
    :param iterations:   Number of iterations for dilatation process

    :return: dilated image
    """
    
    return self.processor.dilate_image(binary_mask=image, iterations=iterations)

  def get_contours(self,
                   image,
                   min_pore_area=None,
                   inverse=False):
    """
    Find the contours and compute areas

    :param image:     Input image
    :param min_area:  Minimum area of the pore to be counted in
    :param inverse:   Set tu True if inverse input image (e.g. for pore analysis) 

    :return: Two List -> list of contours, list of countour's areas 
    """

    return self.processor.find_contours(image, 
                                        min_area=min_pore_area,
                                        inverse=inverse)  

  def _prepare_image(self, image):
    """ Crop out SEM label from the bottom and resize image """

    # Check input
    if image is None:
      raise ValueError("\n_prepare_image: No image on input")

    # Crop out SEM label
    cropped_image = self.processor.crop_label(image)

    # Resize image
    resized_image = self.processor.resize(cropped_image)

    return resized_image

  def _get_micro_meter_per_pixel(self, image, view_field):
    """ Get unit of mircometer per pixel from the image and its view field """

    # Check input
    if image is None:
      raise ValueError("\n_prepare_image: No image on input")
    if view_field is None:
      raise ValueError("\n_prepare_image: No view_field on input")

    # Convert milimeters to micrometers
    micrometers_view_field = view_field * 1000

    # Get image width
    image_width = image.shape[1]

    # Get unit per pixel
    micrometer_per_pixel = micrometers_view_field / image_width

    return micrometer_per_pixel
  
  def _convert_pixels_areas_to_diameter(self,
                                        areas,
                                        micrometer_per_pixel):
    """ Convert pore areas in pexels to diameters in micrometers """

    # Check input
    if not isinstance(areas, list):
      raise ValueError("\n_convert_pixels_areas_to_diameterl: areas must be list")
    
    diameters = []
    for pore_area in areas:
      # Calculate diameter
      diam = 2 * math.sqrt( (pore_area * micrometer_per_pixel)/ math.pi) 

      # Store 
      diameters.append(round(diam, 2))

    return diameters

  def _overlay_pore_mask_to_original(self,
                                     preprocessed_image,
                                     contours,
                                     colored_image,
                                     alpha=None,
                                     pore_color=None):
    """ Overley identified colored pores with original image"""
    # Check input
    if preprocessed_image is None:
      raise ValueError("\n_overlay_pore_mask_to_original: No preprocessed_image on input")
    if contours is None:
      raise ValueError("\n_overlay_pore_mask_to_original: No contours on input")
    if colored_image is None:
      raise ValueError("\n_overlay_pore_mask_to_original: No colored_image on input")
    
    if pore_color is None:
      pore_color = [67, 193, 122]
    if alpha is None:
      alpha = self.DEFAULT_ALPHA

    # Prepare stencil for inverse mask
    stencil = np.zeros(preprocessed_image.shape).astype(preprocessed_image.dtype)
    # Fill stencil with contours 
    cv2.fillPoly(stencil, contours, [255, 255, 255])

    # Get selected contours only from preprocesed image
    contours_image = cv2.bitwise_and(preprocessed_image, stencil)

    # Get mask of contours only
    _, mask = cv2.threshold(contours_image, 1, 255, cv2.THRESH_BINARY)

    # Select pore labels areas 
    indicies = np.where(mask==255)

    # Prepare final image
    colored_image = self._prepare_image(colored_image)
    image_with_pores = np.copy(colored_image)

    # Apply color on image with pores
    image_with_pores[indicies[0], indicies[1], :] = pore_color

    # Get colored image only with pores
    image_with_pores = cv2.bitwise_and(image_with_pores,image_with_pores, mask=mask)

    # Overlay colored pores on original image
    final_image = cv2.addWeighted(colored_image, 1, image_with_pores, alpha, 0)
    # Draw pores lines
    final_image = cv2.drawContours(final_image, contours, -1, pore_color, thickness=2)

    # Calculate porosity (comment out not accurate as a lot of materials still visible) 
    # porosity = round((1-np.count_nonzero(mask) / np.size(mask)) * 100, 1)
    # print(f"Porosity is: {porosity}")
    
    return final_image


<!-- ### Plot & Export Functions -->

In [16]:
ceitec_color = (0.48, 0.76, 0.26)
ceitec_color_hex = ['#7AC143'] 

def plot_hystogram(data, 
                   bins=None, 
                   log_scale=True,
                   grid=True,
                   **fig_kwargs):

  # Prepare the figure settings
  fig_kwargs = fig_kwargs if fig_kwargs else {
      "fig_size": (12, 6), 
      "show_ticks": True, 
      "x_label": "diameter [μm]",
      "y_label": "pores",
      "shade": True
  }
  
  if bins is None:
        bins = 20

  # Create figure object
#   fig, ax = plt.subplots(1, 1, figsize=fig_kwargs.get("fig_size"))

  hist = px.histogram(x=data,
                      nbins=bins,
                      color_discrete_sequence=ceitec_color_hex,
                      log_x=log_scale,
                      labels={'x':fig_kwargs.get("x_label"), 'y':fig_kwargs.get("y_label")},
                      width=1000,
                      height=500)
#   hist.show()

  return hist

def plot_boxplot(data, 
                 grid=True,
                 swarmplot=True,
                 **fig_kwargs):

  # Prepare the figure settings
  fig_kwargs = fig_kwargs if fig_kwargs else {
      "fig_size": (6, 6), 
      "show_ticks": True, 
      "y_label": "diameter [μm]",
      "x_label": "pores",
      "shade": True
  }

  # Create figure object
#   fig, ax = plt.subplots(1, 1, figsize=fig_kwargs.get("fig_size"))
  box = px.box(y=data,
              color_discrete_sequence=ceitec_color_hex,
              labels={'y':fig_kwargs.get("y_label")},
              points="all",
              width=500,
              height=500)
#   box.show()

  return box

<!-- ### Widgets functions -->

In [17]:

pore_analyzator = PoreAnalyzator()
file_flag=False

status_bar = widgets.HTML(value="<b>No Image Uploaded</b>")


def select_files(widget, event, data):
    clear_output()
    root = Tk()
    root.withdraw() # Hide the main window.
    root.call('wm', 'attributes', '.', '-topmost', True) # Raise the root to the top of all windows.
    widget.files = filedialog.askopenfilename(multiple=False) # List of selected files will be set button's file attribute.
    file_flag = True
    print(widget.files) # Print the list of files selected.
    image_path = os.path.normpath(widget.files)
    pore_analyzator.analysis_image = cv2.imread(os.path.join(image_path))
    status_bar.value=f"<b>Image {image_path} ready to process.</b>"

def analyze(low_threshold, min_pore_delimeter, view_field, bins):

    pore_analyzator.extract_pores(image=pore_analyzator.analysis_image,
                                  view_field=view_field,
                                  low_threshold=low_threshold,
                                  min_pore_delimeter=min_pore_delimeter)
    
    number_poures_detected.value=f"{len(pore_analyzator.pore_diameters)}"
    display(number_poures_detected)
    
    # Draw image     
    image = pore_analyzator.pore_image
    height, width = image.shape[:2]
    fig = px.imshow(image, height=height, width=width)
    fig.show()    
      
    change_bins(bins)
    

def change_bins(bins):
    
    hist = plot_hystogram(data=pore_analyzator.pore_diameters,
                               bins=bins,
                               log_scale=False,
                               grid=True)
    
    box = plot_boxplot(pore_analyzator.pore_diameters)
    
    hist.show()
    box.show()
   
def export_results(widget, event, data):
    
    # Select Folder
    root = Tk()
    root.withdraw() # Hide the main window.
    folder_selected  = filedialog.askdirectory() # List of selected files will be set button's file attribute.
    print(folder_selected)
    
    # Get and Store original image
    original_image = pore_analyzator.analysis_image
    cv2.imwrite(f'{folder_selected}\original_image.jpg', original_image)
    
    # Get and Store pore image    
    porouse_image = pore_analyzator.pore_image
    cv2.imwrite(f'{folder_selected}\porouse_image.jpg', porouse_image)
    
    # Get and Store table of pore diameters   
    pore_diameters = pore_analyzator.pore_diameters
    
    df = pd.DataFrame()
  
    # Creating two columns
    df['diameter_[μm]'] = pore_diameters
    df.to_excel(f'{folder_selected}\diameters.xlsx', index = False)
    
    # Get and Store histogram
    histogram = plot_hystogram(data=pore_diameters,
                               bins=bins_changer.value,
                               log_scale=False,
                               grid=True)
    
    path = os.path.join(folder_selected, 'histogram.html')
    histogram.write_html(path)

    # Get and Store boxplot
    box_plot= plot_boxplot(pore_diameters)
    path = os.path.join(folder_selected, 'box_plot.html')
    box_plot.write_html(path)
    
    status_bar.value=f"<b>Results exported in: {os.path.dirname(path)}</b>"
    

style = {'description_width': 'initial'}

number_poures_detected = widgets.Text(value='-',
                                      description='Number of detected pores: ',
                                      disabled=True,
                                      style=style)

view_field = widgets.BoundedFloatText(value=1.00,
                                      min=0.00,
                                      max=100.00,
                                      step=0.01,
                                      description='View Field [mm]:',
                                      placeholder='Set view field of input SEM image',
                                      disabled=False,
                                      style=style)


threshold_slider = widgets.IntSlider(value=pore_analyzator.DEFAULT_LOW_THRESHOLD,
                                     min=0,
                                     max=255,
                                     step=1,
                                     description='Threshold value:',
                                     disabled=False,
                                     continuous_update=False,
                                     orientation='horizontal',
                                     readout=True,
                                     readout_format='d',
                                     style=style,
                                     )

pore_delimeter_pick = widgets.BoundedIntText(value=pore_analyzator.MIN_PORE_DELIMETER,
                                              min=1,
                                              max=2000,
                                              step=1,
                                              description='Minimum pore diameter [μm]:',
                                              placeholder='Set view field of input SEM image',
                                              disabled=False,
                                              style=style)

bins_changer = widgets.BoundedIntText(value=20,
                                      min=1,
                                      max=500,
                                      step=1,
                                      description='Number of bins in histogram:',
                                      disabled=False,
                                      style=style
                                      )


<!-- ### Choose File -->

In [18]:

output = widgets.Output()

@output.capture(clear_output=True,wait=True)    
def run_engine(widget, event, data): 
    if pore_analyzator.analysis_image is None:
        status_bar.value="<b>Upload Image before processing!</b>"
    else:    
        # Show image
        status_bar.value="<b>Processing image...</b>"
        image = pore_analyzator.analysis_image
        height, width = image.shape[:2]
        fig = px.imshow(image, width=width*0.75, height=height*0.75)
        
        fig.show()
        
        widgets.interact(analyze, low_threshold=threshold_slider, min_pore_delimeter=pore_delimeter_pick, view_field=view_field, bins=bins_changer) 
        status_bar.value="<b>Image Processed!</b>"

items_layout = Layout(flex='1 1 auto',
                      width='auto') 

fileselect = v.Btn(color='error', rounded=True, class_='ma-2', children=['Upload Image'], layout=items_layout)
fileselect.on_event('click', select_files)

process_button = v.Btn(color='#7AC143', rounded=True, class_='ma-2', children=['Process Image'], layout=items_layout)
process_button.on_event('click', run_engine)

export_button = v.Btn(color='primary', rounded=True, class_='ma-2', children=['Export Results'], layout=items_layout)
export_button.on_event('click', export_results)

box_layout = Layout(display='flex',
                    flex_flow='row',
                    align_items='stretch',
                    border='solid',
                    width='50%')

box = Box(children=[fileselect, process_button, export_button], layout=box_layout)
display(box)

display(status_bar)

output


Box(children=(Btn(children=['Upload Image'], class_='ma-2', color='error', layout=Layout(flex='1 1 auto', widt…

HTML(value='<b>No Image Uploaded</b>')

Output()